In [1]:
import pandas as pd
import pyodbc

In [2]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [3]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_award;"

# Read sql query
df= pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\4121153418.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(qry, con)


In [4]:
df['funder'].unique()

array(['MRC', 'ISPF', 'EPSRC', 'STFC', 'AHRC', 'BBSRC', 'ESRC', 'NERC',
       'Unknown', 'NC3Rs', 'Infrastructure Fund', 'Disputed', 'HEG',
       'GCRF', 'Newton Fund', 'ISCF', 'UUI', 'SPF', 'FLF', 'FIC',
       'Open Access Block Grant', 'COVID', 'TMF', 'DRI', 'Innovate UK',
       'SiPF', 'Ayrton Fund', 'RE'], dtype=object)

In [5]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_application;"

# Read sql query
df_application = pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\1716727183.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_application = pd.read_sql(qry, con)


In [6]:
# In scope RC: MRC, ISPF (International Science Partnerships Fund), EPSRC, AHRC, BBSRC, ESRC, NERC
# NC3Rs (funded by MRC), GCRF (Global Challenges Reserach Fund), Newton Fund, TMF (Technology Missions Fund),
# Ayrton Fund,

# In scope RE: RE

# Out of scope: Infrastructure Fund, UUI, DRI (Digital Research Infrastructure), Innovate UK

# Unknown: Unknown, Disputed, HEG, ISCF (Industrial Strategy Challenge Fund), SPF (Strategic Priorities Fund), FLF (Future Leaders Fellowships)
# FIC (Fund for International Collaboration), Open Access Block Grant, COVID, Strength in Places Fund

In [7]:
df['administrator'].unique()

array(['MRC', 'BBSRC', 'EPSRC', 'STFC', 'AHRC', 'ESRC', 'NERC', 'NC3Rs',
       'Innovate UK', 'RE'], dtype=object)

In [8]:
df[df['funder'] == 'TMF']['administrator'].unique() # TMF both EPSRC and Innovate UK

array(['EPSRC', 'Innovate UK'], dtype=object)

In [9]:
df[df['funder'] == 'Ayrton Fund']['administrator'].unique()
# This seems contradictory to the website https://www.ukri.org/opportunity/the-ukri-ayrton-challenge-programme-outlines/

array(['Innovate UK'], dtype=object)

In [10]:
df[df['funder'] == 'HEG']['administrator'].unique()

array(['EPSRC', 'Innovate UK'], dtype=object)

In [11]:
df[['application_reference', 'application_source']].duplicated().sum()

df[df[['application_reference', 'application_source']].duplicated(keep=False)].sort_values(by='application_reference')

df[['award_reference', 'application_source']].duplicated().sum()


261

In [12]:
df[df[['award_reference', 'application_source']].duplicated(keep=False)].sort_values(by='award_reference')

df['award_reference'] = df['award_reference'].replace('Unknown', None)

df[df['award_reference'].isna()]

print(df['award_reference'].isna().sum()/len(df)*100)

# Only represents 0.13% of awards

df[df['award_reference'].isna()]['funded_amount'].sum()/df['funded_amount'].sum()*100

# or 0.15% of total funded amount so we can neglect

df_dropna = df[~df['award_reference'].isna()]

0.1261505844657307


In [13]:
# Awards where there are no start or end date (so we cannot tell if they are active or not) represent 1.35% of the total funded value
df_dropna = df_dropna[~(df_dropna['start_date'].isna() & df_dropna['end_date'].isna())]


In [14]:
df_dropna[['award_reference', 'application_source']].duplicated().sum()

0

In [15]:
df_application[['award_reference', 'application_source']].duplicated().sum()

367721

In [16]:
df_application['award_reference'] = df_application['award_reference'].replace('Unknown', None)
df_application.dropna(subset='award_reference', inplace=True)

In [17]:
df_application[['award_reference', 'application_source']].duplicated().sum()

292

In [18]:
df_application[df_application[['award_reference', 'application_source']].duplicated(keep=False)].sort_values(by='award_reference')

df_application.columns

Index(['application_reference', 'award_reference', 'funder', 'decision_owner',
       'application_source', 'administrator', 'opportunity_id',
       'opportunity_name', 'application_category', 'application_status',
       'derived_status', 'decision_date', 'lead_organisation_id',
       'lead_organisation_department_id', 'application_title',
       'amount_applied_for', 'full_economic_cost', 'scheme',
       'rejection_reason', 'date_last_received', 'publish_to_gtr',
       'application_stage', 'application_stage_status', 'joint_proposal',
       'parent_award', 'parent_award_reference', 'redact_abstract_on_gtr',
       'organisation_source', 'international_programme'],
      dtype='object')

In [19]:
df.drop_duplicates(subset=['award_reference', 'application_source'], keep=False, inplace=True)
df_application.drop_duplicates(subset=['award_reference', 'application_source'], keep=False, inplace=True)

In [20]:
data_merge = pd.merge(df, df_application, on=['award_reference', 'application_source'])

In [21]:
print(len(data_merge)/ len(df)*100)

99.86024128789333


In [22]:
data_merge.rename(columns={'application_reference_x':'application_reference', 'funder_x':'funder', 'administrator_x':'administrator'}, inplace=True)

In [23]:
data_merge.drop(columns=['announced_start_date', 'date_last_received', 'rejection_reason', 'publish_to_gtr', 'application_stage', 'application_stage_status', 
                         'joint_proposal', 'parent_award', 'parent_award_reference', 'decision_date', 'derived_status', 'application_status',
                         'redact_abstract_on_gtr'], inplace=True)

In [24]:
data_merge.columns

Index(['application_reference', 'award_reference', 'funder', 'administrator',
       'application_source', 'start_date', 'end_date', 'funded_amount',
       'profiled_amount', 'project_cost', 'termination_date',
       'application_reference_y', 'funder_y', 'decision_owner',
       'administrator_y', 'opportunity_id', 'opportunity_name',
       'application_category', 'lead_organisation_id',
       'lead_organisation_department_id', 'application_title',
       'amount_applied_for', 'full_economic_cost', 'scheme',
       'organisation_source', 'international_programme'],
      dtype='object')

In [25]:
data_merge.drop(columns=['termination_date', 'application_reference_y', 'funder_y', 'administrator_y', 'lead_organisation_department_id'], inplace=True)

In [26]:
data_merge['application_category'].unique()

array(['Research and Innovation', 'Fellowship', 'Training Grant',
       'Research Grant', 'Intramural', 'P&Cs', 'Third Party Grant',
       'Other Grant', 'Studentship', None, 'Off System', 'Other', 'Unit',
       'Collaborative R&D', 'EU-Funded', 'BEIS-Funded Programmes',
       'Feasibility Studies', 'Study', 'Responsive Strategy and Planning',
       'Small Business Research Initiative', 'CR&D Bilateral', 'Centres',
       'GRD Proof of Concept', 'Knowledge Transfer Network', 'Missions',
       'Demonstrator', 'SME Support', 'Grant for R&D',
       'GRD Development of Prototype', 'Innovation Loans', 'Fast Track',
       'GRD Proof of Market', 'Legacy RDA Grant for R&D', 'Vouchers',
       'Knowledge Transfer Partnership', 'Special Interest Group',
       'Legacy RDA Collaborative R&D', 'Launchpad', 'Procurement',
       'Large Project', 'Legacy Department of Trade & Industry',
       'European Enterprise Network', 'Investment Accelerator',
       'Memorandum of Agreement',
       '

In [27]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_spend;"

# Read sql query
df_spend = pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\4159353175.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_spend = pd.read_sql(qry, con)


In [28]:
df_spend['paymentdate_financial_year'].unique()

array(['2005/06', '2006/07', '2007/08', '2002/03', '2003/04', '2004/05',
       '2001/02', '2008/09', '2009/10', '2010/11', '2000/01', '2011/12',
       '2012/13', '2013/14', '2014/15', '2015/16', '2016/17', '2017/18',
       '2018/19', '2019/20', '2020/21', '2021/22', '2022/23', '2023/24',
       '2024/25', 'Unknown', '1989/90', '1990/91', '1991/92', '1992/93',
       '1993/94', '1994/95', '1995/96', '1996/97', '1997/98', '1998/99',
       '1999/00', '1988/89', '2025/26', '2026/27', '2027/28', '2028/29'],
      dtype=object)

In [29]:
df_spend_2324 = df_spend.loc[df_spend['paymentdate_financial_year'] == '2023/24']


In [30]:
df_spend_2324.drop(columns=['transfer_reference'], inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\4199183093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spend_2324.drop(columns=['transfer_reference'], inplace=True)


In [31]:
data_merge[['award_reference', 'application_source']].duplicated().sum()

0

In [32]:
data_w_spend = pd.merge(data_merge, df_spend_2324, on=['award_reference', 'application_source'], how='outer', indicator=True)

In [33]:
data_w_spend[data_w_spend['_merge'] == 'left_only'].sort_values(by='start_date', ascending=False)

,application_reference_x,award_reference,funder_x,administrator_x,application_source,start_date,end_date,funded_amount,profiled_amount,project_cost,...,paymentdate,paymentdate_financial_year,PaymentQuarterID,ExpenditureType,amount,type,funder_y,administrator_y,payment_type,_merge
417643,20453,MR/Z505626/1,MRC,MRC,TFS,2027-04-01,2029-09-30,1341726.650,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
458361,NE/Y001338/1,NE/Y001338/1,NERC,NERC,Siebel,2026-06-01,2029-05-31,502872.598,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
458329,NE/Y000811/1,NE/Y000811/1,NERC,NERC,Siebel,2026-06-01,2029-05-31,429536.560,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
492168,ST/Y004795/1,ST/Y004795/1,STFC,STFC,Siebel,2026-04-01,2028-03-31,85594.198,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
417589,23067,MR/Z505031/1,MRC,MRC,TFS,2026-02-01,2029-01-31,379182.640,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461314,8,NGT8,AHRC,AHRC,TFS,NaT,NaT,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
461315,856,NGT856,AHRC,AHRC,TFS,NaT,NaT,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
461316,880,NGT880,AHRC,AHRC,TFS,NaT,NaT,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
461317,976,NGT976,AHRC,AHRC,TFS,NaT,NaT,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [34]:
data_w_spend[data_w_spend['_merge'] == 'right_only']['amount'].sum()

586237.97

In [35]:
data_w_spend[data_w_spend['_merge'] == 'right_only'] # funded_amount is null

,application_reference_x,award_reference,funder_x,administrator_x,application_source,start_date,end_date,funded_amount,profiled_amount,project_cost,...,paymentdate,paymentdate_financial_year,PaymentQuarterID,ExpenditureType,amount,type,funder_y,administrator_y,payment_type,_merge
10228,NaN,10030366,NaN,NaN,IUK - IFS,NaT,NaT,NaN,NaN,NaN,...,2023-10-31,2023/24,2023/24 Q3,Unknown,30937.86,None,Innovate UK,Innovate UK,None,right_only
10229,NaN,10030366,NaN,NaN,IUK - IFS,NaT,NaT,NaN,NaN,NaN,...,2024-01-31,2023/24,2023/24 Q4,Unknown,34362.72,None,Innovate UK,Innovate UK,None,right_only
74829,NaN,900263,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2023-09-30,2023/24,2023/24 Q2,Unknown,16807.89,None,Innovate UK,Innovate UK,None,right_only
74830,NaN,900263,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2023-12-31,2023/24,2023/24 Q3,Unknown,15401.06,None,Innovate UK,Innovate UK,None,right_only
74831,NaN,900263,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2024-03-31,2023/24,2023/24 Q4,Unknown,22686.55,None,Innovate UK,Innovate UK,None,right_only
74832,NaN,900263,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2023-09-30,2023/24,2023/24 Q2,Unknown,10734.66,None,Innovate UK,Innovate UK,None,right_only
74833,NaN,900263,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2023-12-31,2023/24,2023/24 Q3,Unknown,51824.42,None,Innovate UK,Innovate UK,None,right_only
74834,NaN,900263,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2024-03-31,2023/24,2023/24 Q4,Unknown,68223.26,None,Innovate UK,Innovate UK,None,right_only
74835,NaN,900264,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2023-09-30,2023/24,2023/24 Q2,Unknown,16284.14,None,Innovate UK,Innovate UK,None,right_only
74836,NaN,900264,NaN,NaN,IUK - Legacy Applications,NaT,NaT,NaN,NaN,NaN,...,2023-12-31,2023/24,2023/24 Q3,Unknown,35022.15,None,Innovate UK,Innovate UK,None,right_only


In [36]:
data_both = data_w_spend[data_w_spend['_merge'] == 'both']

In [37]:
data_both['amount'].sum()

4941106860.0368

In [38]:
data_both.rename(columns={'application_reference_x':'application_reference', 'funder_x':'funder', 'administrator_x':'administrator'}, inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\459481331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_both.rename(columns={'application_reference_x':'application_reference', 'funder_x':'funder', 'administrator_x':'administrator'}, inplace=True)


In [39]:
data_both.drop(columns=['_merge', 'profiled_amount', 'project_cost', 'opportunity_id', 'application_reference_y', 'international_programme'], inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\2982597896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_both.drop(columns=['_merge', 'profiled_amount', 'project_cost', 'opportunity_id', 'application_reference_y', 'international_programme'], inplace=True)


In [40]:
data_both.columns

Index(['application_reference', 'award_reference', 'funder', 'administrator',
       'application_source', 'start_date', 'end_date', 'funded_amount',
       'decision_owner', 'opportunity_name', 'application_category',
       'lead_organisation_id', 'application_title', 'amount_applied_for',
       'full_economic_cost', 'scheme', 'organisation_source', 'paymentdate',
       'paymentdate_financial_year', 'PaymentQuarterID', 'ExpenditureType',
       'amount', 'type', 'funder_y', 'administrator_y', 'payment_type'],
      dtype='object')

In [41]:
data_both.drop(columns=['administrator_y', 'funder_y', 'paymentdate', 'PaymentQuarterID', 'application_title',
                        'amount_applied_for', 'full_economic_cost'], inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\3005258166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_both.drop(columns=['administrator_y', 'funder_y', 'paymentdate', 'PaymentQuarterID', 'application_title',


In [42]:
data_both.drop(columns=['payment_type', 'type'], inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\3769455728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_both.drop(columns=['payment_type', 'type'], inplace=True)


In [43]:
data_both[['funder', 'administrator', 'decision_owner', 'application_category']].drop_duplicates().to_clipboard(index=False)

In [44]:
data_both['amount'].sum() # £4.9 Bn

4941106860.0368

In [45]:
data_both[data_both['funder'] == 'COVID']['amount'].sum()

8309584.364

In [46]:
data_both[(data_both['funder'] == 'COVID')]['opportunity_name'].unique()

array(['COVID 19 Rapid Response', 'COVID19 Ext Allocation V2',
       'COVID-19: Long COVID Call', 'COVID-19 RRR Open Call',
       'Global Effort on COVID-19', 'SARS CoV 2',
       'NCS COVID19 BIG DATA 2021'], dtype=object)

In [47]:
data_both['ExpenditureType'].unique()

array(['Unknown', 'Exception - Other Costs', 'Exception - Staff',
       'DI - Other Costs', 'DI - T&S', 'DI - Staff', 'DA - Investigators',
       'Exception - Equipment', 'DA - Other Directly Allocated',
       'Indirect - Indirect Costs', 'DA - Estate Costs', 'DI - Equipment',
       'DI - Absence', 'Total - DSA', 'Total - Student Stipend',
       'Total - Fees', 'Total - Other Costs', 'Total - RTSG',
       'Total - Absence', 'Total - Fieldwork', 'Total - Robert Skills',
       'Total - T&S', 'Total - Consumables', 'Total - Staff',
       'Exception - T&S', 'Total - OffSystem', 'Total - Equipment',
       'Total - Indirect Costs', 'Total - Large Capital'], dtype=object)

In [48]:
data_both[['funder', 'application_category', 'ExpenditureType']].drop_duplicates()

,funder,application_category,ExpenditureType
2174,Innovate UK,Collaborative R&D,Unknown
2177,ISCF,CR&D Bilateral,Unknown
2189,Innovate UK,Knowledge Transfer Partnership,Unknown
2251,Innovate UK,Study,Unknown
2257,ISCF,Feasibility Studies,Unknown
...,...,...,...
491207,STFC,Research and Innovation,DA - Other Directly Allocated
491212,STFC,Research and Innovation,DI - Other Costs
491217,STFC,Research and Innovation,DI - Staff
491222,STFC,Research and Innovation,DI - T&S


In [49]:
ds_mapping = pd.read_excel('..//OneDrive - UKRI//Dual Support Categories.xlsx')

In [50]:
data_w_ds = pd.merge(
    data_both,
    ds_mapping[['funder', 'administrator', 'decision_owner', 'application_category', 'DS']],
    on=['funder', 'administrator', 'decision_owner', 'application_category']
    )

In [51]:
data_w_ds

print(len(data_both))
print(len(data_w_ds))

310829
310829


In [52]:
data_w_ds['ExpenditureType'].str.contains('Equipment')

0         False
1         False
2         False
3         False
4         False
          ...  
310824    False
310825    False
310826    False
310827     True
310828     True
Name: ExpenditureType, Length: 310829, dtype: bool

In [53]:
data_w_ds[(data_w_ds['DS'] == 'RC') & data_w_ds['ExpenditureType'].str.contains('Equipment')]['award_reference'].unique()

array(['AH/L011727/1', 'AH/N00664X/1', 'AH/N010140/1', ...,
       'ST/Y006135/1', 'ST/Y006143/1', 'ST/Z000084/1'], dtype=object)

In [54]:
data_w_ds.loc[data_w_ds['ExpenditureType'].str.contains('Equipment'), 'DS'] = 'Exclude'
data_w_ds.loc[data_w_ds['ExpenditureType'].str.contains('Capital'), 'DS'] = 'Exclude'

In [55]:
data_w_ds.groupby('DS').sum(numeric_only=True)

,funded_amount,amount
DS,,
Exclude,1.706385e+11,2.210966e+09
RC,3.191392e+11,2.646206e+09
Unknown,9.749007e+07,8.393448e+07


In [56]:
# eg. NERC Off System spend 260M according to sustainability returns
# Unknown 84 Million, Excluded 2.2Bn and RC in scope 2.6Bn

In [57]:
2.1/2.3

0.9130434782608696

In [58]:
64/100

0.64

In [59]:
84/2600

0.03230769230769231

In [60]:
data_w_ds['application_source'].unique()

array(['IUK - IFS', 'IUK - Legacy Applications', 'IUK - KTP',
       'IUK - EU Funded', 'IUK - SBRI', 'Siebel', 'TFS', 'Offsystem'],
      dtype=object)

In [61]:
data_w_ds[data_w_ds['application_source'] == 'Offsystem']['amount'].sum()

286967552.05

In [62]:
data_w_ds.groupby(['DS', 'application_source']).sum(numeric_only=True)

funded_amount        amount
DS      application_source                                    
Exclude IUK - EU Funded             4.346026e+07  3.403815e+06
        IUK - IFS                   6.393452e+10  1.218005e+09
        IUK - KTP                   5.123945e+07  4.089798e+06
        IUK - Legacy Applications   9.134722e+10  7.308103e+08
        IUK - SBRI                  3.559132e+06  3.184158e+05
        Offsystem                   2.541528e+06  1.491646e+06
        Siebel                      1.522373e+10  2.518602e+08
        TFS                         3.222036e+07  9.872006e+05
RC      Offsystem                   5.976107e+08  2.015414e+08
        Siebel                      3.177586e+11  2.431201e+09
        TFS                         7.830291e+08  1.346364e+07
Unknown Offsystem                   9.749007e+07  8.393448e+07

In [63]:
data_w_ds[(data_w_ds['DS'] == 'RC') & (data_w_ds['application_source'] != 'Offsystem')]['amount'].sum()

2444664624.9922

In [ ]:
# 2.4 Bn RC dual support and not Off system

# Off system dual support RC is the institutes
# 

In [64]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2024_10_04_ssd_organisation;"

# Read sql query
df_organisation = pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_15928\1599731779.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_organisation = pd.read_sql(qry, con)


In [67]:
df_select = df_organisation[['organisation_id', 'organisation_name', 'organisation_source']]


In [68]:
df_organisation[['organisation_id', 'organisation_source']].duplicated().sum()

0

In [70]:
data_w_org = pd.merge(data_w_ds, df_select, left_on=['lead_organisation_id', 'organisation_source'],
                      right_on=['organisation_id', 'organisation_source'])

In [71]:
print(len(data_w_ds))
print(len(data_w_org))


310829
310829


In [75]:
data_w_org[data_w_org['organisation_name'].str.contains('British Antarctic')]['organisation_name'].unique()

array(['NERC British Antarctic Survey'], dtype=object)

In [77]:
data_w_org[data_w_org['organisation_name'].str.contains('British Antarctic')].groupby(['DS', 'application_source']).sum(numeric_only=True)

funded_amount      amount
DS      application_source                           
Exclude Siebel               7.441292e+06    82928.98
RC      Siebel               6.036610e+08  5439063.38
        TFS                  1.093147e+07   109720.13

In [78]:
data_w_org[data_w_org['organisation_name'].str.contains('British Antarctic')].groupby(['DS', 'application_category', 'application_source']).sum(numeric_only=True)

funded_amount      amount
DS      application_category    application_source                           
Exclude Fellowship              Siebel               1.032502e+06     5817.00
        Research Grant          Siebel               6.408791e+06    77111.98
RC      Fellowship              Siebel               7.529438e+07   722164.22
        Research Grant          Siebel               5.281241e+08  4676472.16
        Research and Innovation TFS                  1.093147e+07   109720.13
        Training Grant          Siebel               2.425620e+05    40427.00

In [80]:
data_w_org[data_w_org['organisation_name'].str.contains('Hydrology')]['organisation_name'].unique()

array(['UK Centre for Ecology & Hydrology'], dtype=object)

In [84]:
data_w_org[data_w_org['organisation_name'].str.contains('Turing')].groupby(['DS', 'application_source']).sum(numeric_only=True)

funded_amount        amount
DS      application_source                             
Exclude Siebel               1.619304e+07  4.275000e+04
RC      Siebel               7.896944e+08  1.577897e+07
        TFS                  1.008674e+07  3.272235e+05